# Entra Agent ID Sidecar Weather Agent

This notebook mirrors the weather agent flow in langgraph_test.ipynb, but retrieves Agent Identity tokens via the Microsoft Entra sidecar for each tool call.

**What you will do:**
- Configure and run the sidecar container
- Request Agent Identity tokens via sidecar
- Run the mock weather agent (single-shot + streaming)

## Sidecar setup (one-time)

Follow the SIDECAR-GUIDE.md steps in the 3P-Agent-ID-Demo repo.

### 1) Configure the sidecar .env
```powershell
cd sidecar
Copy-Item .env.example .env
# Edit .env with values from Start-EntraAgentIDWorkflow:
# TENANT_ID=<your-tenant-id>
# BLUEPRINT_APP_ID=<blueprint-app-id>
# AGENT_CLIENT_ID=<agent-app-id>
# BLUEPRINT_CLIENT_SECRET=<blueprint-secret>
```

### 2) Start the sidecar
```powershell
docker-compose up -d
Invoke-RestMethod -Uri "http://localhost:5001/healthz"  # Expect: Healthy
```

In [1]:
# Optional: install dependencies
#%pip install -q langchain-openai langchain-core python-dotenv requests
import asyncio
import os
from random import randint
from typing import Annotated
import requests
from dotenv import load_dotenv
from pydantic import Field

from langchain_core.tools import tool
from langchain_openai import AzureChatOpenAI
from langchain.agents import create_agent

load_dotenv()

REQUIRED_ENV = [
    "ENTRA_AGENT_APP_ID",
    "AZURE_OPENAI_ENDPOINT",
    "AZURE_OPENAI_API_KEY",
    "AZURE_OPENAI_API_VERSION",
    "AZURE_OPENAI_DEPLOYMENT",
]

missing = [key for key in REQUIRED_ENV if not os.getenv(key)]
if missing:
    raise EnvironmentError(f"Missing environment variables: {', '.join(missing)}")

AGENT_APP_ID = os.getenv("ENTRA_AGENT_APP_ID")
SIDECAR_BASE_URL = os.getenv("SIDECAR_BASE_URL", "http://localhost:5001")

In [2]:
def get_agent_token_from_sidecar():
    url = f"{SIDECAR_BASE_URL}/AuthorizationHeaderUnauthenticated/graph"
    resp = requests.get(url, params={"AgentIdentity": AGENT_APP_ID})
    resp.raise_for_status()
    return resp.json()["authorizationHeader"]  # Includes 'Bearer ' prefix

In [3]:
@tool
def get_weather(
    location: Annotated[str, Field(description="The location to get the weather for.")]
) -> str:
    """Get the weather for a given location (mock)."""
    _ = get_agent_token_from_sidecar()  # Sidecar retrieves Agent Identity token
    conditions = ["sunny", "cloudy", "rainy", "stormy"]
    return f"The weather in {location} is {conditions[randint(0, 3)]} with a high of {randint(10, 30)}°C."

In [4]:
llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
    temperature=0,
 )

agent = create_agent(
    model=llm,
    tools=[get_weather],
    system_prompt="You are a helpful weather agent.",
)

In [5]:
query = "What's the weather like in Seattle?"
result = agent.invoke({"messages": [("user", query)]})
result["messages"][-1].content

HTTPError: 401 Client Error: Unauthorized for url: http://localhost:5001/AuthorizationHeaderUnauthenticated/graph?AgentIdentity=blah

In [6]:
async def stream_agent(query: str):
    print("Agent:", end=" ", flush=True)
    async for event in agent.astream_events(
        {"messages": [("user", query)]},
        version="v2",
    ):
        if event["event"] == "on_chat_model_stream":
            chunk = event["data"]["chunk"]
            if getattr(chunk, "content", None):
                print(chunk.content, end="", flush=True)
    print()

await stream_agent("What's the weather like in Portland?")

Agent: Sure! Let me check the weather for Portland.

HTTPError: 401 Client Error: Unauthorized for url: http://localhost:5001/AuthorizationHeaderUnauthenticated/graph?AgentIdentity=blah

## Notes
- If the sidecar returns 403, verify the Agent Identity has required permissions and wait for propagation.
- If you see 404, confirm the sidecar is healthy at `http://localhost:5001/healthz`.
- This uses the token-only pattern (`/AuthorizationHeaderUnauthenticated`) so your app controls HTTP calls.